How do "real users" (without the fake ones) fill out their profile data?

In [76]:
import pandas as pd
from matplotlib import pyplot as plt
import pymysql
% matplotlib inline

connection = pymysql.connect(host='kidup-production.c5gkrouylqmw.eu-central-1.rds.amazonaws.com',
                             user='kidup_148161251',
                             password='knop0602R',
                             db='kidup',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
df = pd.read_sql_query("""select * from profile WHERE user_id not in (1,4,7,142,3940)
and !(user_id > 4900 and user_id < 5248)""", connection)

plot_df = {}
plot_df['without_first_name'] = len(df[df.first_name == ''])
plot_df['without_last_name'] = len(df[df.last_name == ''])
plot_df['without_description'] = len(df[(df.description.str.len() < 6)])
plot_df['without_img'] = len(df[(df.img == 'kidup/user/default-face.jpg')])
plot_df['without_phone'] = len(df[pd.isnull(df.phone_number)])
plot_df['email_verified'] = len(df[df.email_verified == 1])
plot_df['phone_verified'] = len(df[df.phone_verified == 1])
plot_df['all'] = len(df)
soc_accounts = pd.read_sql_query("""select count(1) as cnt from social_account """, connection)
plot_df['social_accounts'] = soc_accounts.cnt[0]

df_tmp = pd.DataFrame(plot_df, index=[0])
df_tmp

,all,email_verified,phone_verified,social_accounts,without_description,without_first_name,without_img,without_last_name,without_phone
0,1284,26,9,925,294,176,450,176,1253


In [78]:
plot_df2 = {}

for i, data in enumerate(plot_df):
    plot_df2[data] = int(plot_df[data])/1284
 
df_tmp = pd.DataFrame(plot_df2, index=[0])
df_tmp

,all,email_verified,phone_verified,social_accounts,without_description,without_first_name,without_img,without_last_name,without_phone
0,1,0.020249,0.007009,0.720405,0.228972,0.137072,0.350467,0.137072,0.975857


Interestingly, users fill out a lot of information, especially the information that we force them to provide in the post-registration. A lot of the signups are social, and a lot of users have an image (which is automated when signing up over facebook).


### Is it possible to seperate non-Facebook user behaviour from Facebook user behaviour? (If so, then we could target this behaviour to improve the traffic streams)

In [88]:
df = pd.read_sql_query("""select count(1) as cnt from tracking_event
 WHERE user_id in (select distinct user_id from social_account)
and type='page_view'""", connection)
df2 = pd.read_sql_query("""select count(1) as cnt from tracking_event
 WHERE user_id not in (select distinct user_id from social_account)
and type='page_view'""", connection)
soc_acc = pd.read_sql_query("""select count(1) as cnt from social_account""", connection)

print((df.cnt[0])/(soc_acc.cnt[0]), "Page views per user that signed up via facebook")
print((df2.cnt[0])/(1284 - soc_acc.cnt[0]), "Page views per user that signed up via email")

7.41513513514 Page views per user that signed up via facebook
8.44846796657 Page views per user that signed up via email
